In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn import preprocessing
from keras import layers
from keras.models import Sequential
from keras.utils import plot_model
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
data_path = 'D:\\jupyter files\\data_waiting_time_project\\preprocess_data\\'

In [3]:
data_part1 = pd.read_csv(data_path + 'data_part1.csv', index_col = 0)
data_part2 = pd.read_csv(data_path + 'data_part2.csv', index_col = 0)

C:\Users\WENDY\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
# 把字符串转成datetime
def str_to_Datetime(st):
    dt = datetime.datetime.strptime(st, "%Y-%m-%d %H:%M:%S")
    return dt

In [5]:
feature_num = ['Scheduled_duration', 'Actual_duration',
               'age', 'TreatmentTime_total', 'ImagesTaken_total',
               'MU_total', 'MUCoeff_total', 'Interval_scheduled']

# RadiationId
feature_cate = ['dxt_AliasName', 'Sex', 'AliasSerNum',
                'month', 'week', 'hour', 'DoctorSerNum', 
                'TreatmentOrientation', 'FractionNumber',
                'UserName', 'CourseId', 'ResourceSerNum']

In [6]:
# 因为需要进行onehot encoding，所以在拼接数据之前，先进行数据格式的处理
for col in feature_cate:
    try:
        data_part1[col].fillna('Unknown', inplace = True)
        data_part1[col] = data_part1[col].astype(str)
    except:
        pass
    
    try:
        data_part2[col].fillna('Unknown', inplace = True)
        data_part2[col] = data_part2[col].astype(str)
    except:
        pass

for col in feature_num:
    try:
        data_part1.fillna(0, inplace = True)
    except:
        pass
    try:
        data_part2.fillna(0, inplace = True)
    except:
        pass
    
# data_num = log1p(data_num)

In [7]:
# one hot encoder
label_encoder_dxt_AliasName = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_dxt_AliasName.fit(data_part1.dxt_AliasName.values.reshape(-1, 1))

label_encoder_Sex = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_Sex.fit(data_part1.Sex.values.reshape(-1, 1))

label_encoder_AliasSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_AliasSerNum.fit(data_part1.AliasSerNum.values.reshape(-1, 1))

label_encoder_month = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_month.fit(data_part1.month.values.reshape(-1, 1))

label_encoder_week = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_week.fit(data_part1.week.values.reshape(-1, 1))

label_encoder_hour = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_hour.fit(data_part1.hour.values.reshape(-1, 1))

label_encoder_DoctorSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_DoctorSerNum.fit(data_part1.DoctorSerNum.values.reshape(-1, 1))

label_encoder_TreatmentOrientation = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_TreatmentOrientation.fit(data_part1.TreatmentOrientation.values.reshape(-1, 1))

label_encoder_FractionNumber = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_FractionNumber.fit(data_part2.FractionNumber.values.reshape(-1, 1))

label_encoder_UserName = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_UserName.fit(data_part2.UserName.values.reshape(-1, 1))

label_encoder_CourseId = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_CourseId.fit(data_part2.CourseId.values.reshape(-1, 1))

label_encoder_ResourceSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
label_encoder_ResourceSerNum.fit(data_part2.ResourceSerNum.values.reshape(-1, 1))

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=False)

==============================================================

#### categorical feature 不同取值的个数

In [8]:
# feature_num = ['Scheduled_duration', 'Actual_duration', 'Actual_duration',
#                'age', 'TreatmentTime_total', 'ImagesTaken_total',
#                'MU_total', 'MUCoeff_total']

# # RadiationId
# feature_cate = ['dxt_AliasName', 'Sex', 'AliasSerNum',
#                 'month', 'week', 'hour', 'DoctorSerNum', 
#                 'TreatmentOrientation', 'FractionNumber',
#                 'UserName', 'CourseId', 'ResourceSerNum']

# feature_count1 = pd.DataFrame({})
# for col in feature_cate:
#     try:
#         n = len(data_part1[col].unique())
#         feature_count1[col] = [n]
#     except:
#         pass
    
# feature_count2 = pd.DataFrame({})
# for col in feature_cate:
#     try:
#         n = len(data_part2[col].unique())
#         feature_count2[col] = [n]
#     except:
#         pass

==============================================================

In [9]:
data_part1_grouped = data_part1.iloc[: 10000].groupby('AppointmentSerNum')

appt_one_list = []
appt_more_list = []
for appt in data_part1_grouped.groups.keys():
    sample = data_part1_grouped.get_group(appt)
    if sample.shape[0] == 1:
        appt_one_list.append(appt)
print(f'{len(appt_one_list)} with one appointment')

appt_more_list = [i for i in data_part1_grouped.groups.keys() if i not in appt_one_list]
print(f'{len(appt_more_list)} with more appointment')


# key_list = list(data_part1_grouped.groups.keys())
data_part1_new = pd.DataFrame({})
for i in range(len(appt_more_list)):
    if (i + 1) % 500 == 0:
        print(f'\n{i+1} appointment data')
        
    sample = data_part1_grouped.get_group(appt_more_list[i])
    new_appt = pd.DataFrame({
        'PatientSerNum': sample.PatientSerNum.tolist()[0],
        'AppointmentSerNum': sample.AppointmentSerNum.tolist()[0],
        'Sex': sample.Sex.tolist()[0],
        'age': sample.age.tolist()[0],
        'DoctorSerNum': sample.DoctorSerNum.tolist()[0],
        'month': sample.month.tolist()[0],
        'date': sample.date.tolist()[0],
        'week': sample.week.tolist()[0],
        'hour': sample.hour.tolist()[0],
        'Scheduled_duration': sample.Scheduled_duration.tolist()[0],
        'Actual_duration': sample.Actual_duration.tolist()[0],
        'ScheduledStartTime': sample.ScheduledStartTime.tolist()[0],
        'ScheduledEndTime': sample.ScheduledEndTime.tolist()[0],
        'ActualStartDate': sample.ActualStartDate.tolist()[0],
        'ActualEndDate': sample.ActualEndDate.tolist()[0],
        
        'dxt_AliasName': [list(set(sample.dxt_AliasName.tolist()))], 
        'AliasSerNum': [list(set(sample.AliasSerNum.tolist()))], 
        'CourseSerNum': [list(set(sample.CourseSerNum.tolist()))], 
        'PlanSerNum': [list(set(sample.PlanSerNum.tolist()))], 
        'TreatmentOrientation': [list(set(sample.TreatmentOrientation.tolist()))],
        
    })
    
    data_part1_new = pd.concat([data_part1_new, new_appt], axis = 0)
data_part1_new.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime'], inplace = True)
data_part1_new.reset_index(drop = True, inplace = True)

457 with one appointment
1900 with more appointment

500 appointment data

1000 appointment data

1500 appointment data


In [10]:
data_part2_grouped = data_part2.iloc[: 10000].groupby(['PatientSerNum', 'date'])

treat_one_list = []
treat_more_list = []
for k in data_part2_grouped.groups.keys():
    sample = data_part2_grouped.get_group(k)
    if sample.shape[0] == 1:
        treat_one_list.append(appt)
print(f'{len(treat_one_list)} with one treatment')

treat_more_list = [i for i in data_part2_grouped.groups.keys() if i not in treat_one_list]
print(f'{len(treat_more_list)} with more treatment')

# key_list = list(data_part2_grouped.groups.keys())
data_part2_new = pd.DataFrame({})
for i in range(len(treat_more_list)):
    if (i+1) % 500 == 0:
        print(f'\n{i+1} treatment data')
        
    sample = data_part2_grouped.get_group(treat_more_list[i])
    try:
        new_treat = pd.DataFrame({
            'PatientSerNum': sample.PatientSerNum.tolist()[0],
            'FractionNumber': sample.FractionNumber.tolist()[0],
            'date': sample.date.tolist()[0],

            'UserName': [list(set(sample.UserName.tolist()))], 
            'RadiationSerNum': [list(set(sample.RadiationSerNum.tolist()))], 
            'RadiationId': [list(set(sample.RadiationId.tolist()))], 
            'ResourceSerNum': [list(set(sample.ResourceSerNum.tolist()))], 
            'CourseId': [list(set(sample.CourseId.tolist()))],

            'ImagesTaken_total': sum(sample.ImagesTaken.tolist()),
            'MU_total': sum(sample.MU.tolist()),
            'MUCoeff_total': sum(sample.MUCoeff.tolist()),
            'TreatmentTime_total': sum(sample.TreatmentTime.tolist()),
        })
        data_part2_new = pd.concat([data_part2_new, new_treat], axis = 0)
    except:
        print(treat_more_list[i])
    
    
data_part2_new.sort_values(by = ['PatientSerNum', 'date', 'FractionNumber'], inplace = True)
data_part2_new.reset_index(drop = True, inplace = True)

16 with one treatment
3118 with more treatment

500 treatment data

1000 treatment data

1500 treatment data

2000 treatment data

2500 treatment data

3000 treatment data


In [11]:
data_part1_new.head()
print(f'The shape of data_part1_new is {data_part1_new.shape}')
print(f'The shape of data_part1_new is {data_part2_new.shape}')

DATA = pd.merge(data_part1_new, data_part2_new, on = ['PatientSerNum', 'date'], how = 'inner')
DATA.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
print(f'The shape of data is {DATA.shape}')

The shape of data_part1_new is (1900, 20)
The shape of data_part1_new is (3118, 12)
The shape of data is (584, 30)


In [12]:
DATA_grouped = DATA.groupby('PatientSerNum')

series_data = []
for pat in DATA_grouped.groups.keys():
    if DATA_grouped.get_group(pat).shape[0] > 2:
        series_data.append(pat)
print(f'series data {len(series_data)}')

series_data = pd.DataFrame({'PatientSerNum': series_data})
series_data = pd.merge(series_data, DATA, on = 'PatientSerNum', how = 'inner')
series_data.sort_values(by = ['PatientSerNum', 'ScheduledStartTime'], inplace = True)

series data 47


In [13]:
series_data_grouped = series_data.groupby('PatientSerNum')
maxlen = series_data_grouped.count().AppointmentSerNum.max()
print(f'The max length of patient is {maxlen}')

The max length of patient is 35


#### samlpe train

In [31]:
appt_maxlen = 35
feature_len = 438

In [14]:
train_sample = series_data_grouped.get_group(209)

In [15]:
label_sample = train_sample.Actual_duration.iloc[-1]
print(f'label sample: {label_sample}')

# 最后一个appointment 为我们需要预测的真实治疗时长所对应的appointment
train_sample.Actual_duration.iloc[-1] = 0

label sample: 19.0


C:\Users\WENDY\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [16]:
# 因为存在相隔很远的两次预约，因此，构造特征Interval_scheduled 来度量两次预约之间的距离
train_sample['Last_ScheduledStartTime'] = train_sample.ScheduledStartTime.shift(periods = 1,
                                                                                fill_value = train_sample.ScheduledStartTime.iloc[0])
train_sample['Last_ScheduledStartTime'] = train_sample.Last_ScheduledStartTime.apply(lambda x: str_to_Datetime(x))
train_sample['ScheduledStartTime'] = train_sample.ScheduledStartTime.apply(lambda x: str_to_Datetime(x))
train_sample['Interval_scheduled'] = train_sample.apply(lambda x: 
                                                                  (x.ScheduledStartTime - x.Last_ScheduledStartTime).days, 
                                                                  axis = 1)

train_num = train_sample[feature_num]

C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [17]:
encode_cate = pd.DataFrame({})

encode_cate['Sex'] = train_sample['Sex'].apply(lambda x: sum(label_encoder_Sex.transform(np.array(x).reshape(-1,1))))

encode_cate['dxt_AliasName'] = train_sample['dxt_AliasName'].apply(lambda x: sum(label_encoder_dxt_AliasName.transform(np.array(x).reshape(-1,1))))
train_x = np.vstack(encode_cate.dxt_AliasName.tolist())

encode_cate['AliasSerNum'] = train_sample['AliasSerNum'].apply(lambda x: sum(label_encoder_AliasSerNum.transform(np.array(x).reshape(-1,1))))
train_x = np.hstack(
    (train_x, np.vstack(encode_cate.AliasSerNum.tolist()))
)

encode_cate['month'] = train_sample['month'].apply(lambda x: sum(label_encoder_month.transform(np.array(x).reshape(-1,1))))
train_x = np.hstack(
    (train_x, np.vstack(encode_cate.month.tolist()))
)

encode_cate['week'] = train_sample['week'].apply(lambda x: sum(label_encoder_week.transform(np.array(x).reshape(-1,1))))
train_x = np.hstack(
    (train_x, np.vstack(encode_cate.week.tolist()))
)

encode_cate['hour'] = train_sample['hour'].apply(lambda x: sum(label_encoder_hour.transform(np.array(x).reshape(-1,1))))
train_x = np.hstack(
    (train_x, np.vstack(encode_cate.hour.tolist()))
)

encode_cate['DoctorSerNum'] = train_sample['DoctorSerNum'].apply(lambda x: sum(label_encoder_DoctorSerNum.transform(np.array(x).reshape(-1,1))))
train_x = np.hstack(
    (train_x, np.vstack(encode_cate.DoctorSerNum.tolist()))
)

encode_cate['TreatmentOrientation'] = train_sample['TreatmentOrientation'].apply(lambda x: sum(label_encoder_TreatmentOrientation.transform(np.array(x).reshape(-1,1))))
train_x = np.hstack(
    (train_x, np.vstack(encode_cate.TreatmentOrientation.tolist()))
)

encode_cate['FractionNumber'] = train_sample['FractionNumber'].apply(lambda x: sum(label_encoder_FractionNumber.transform(np.array(x).reshape(-1,1))))
train_x = np.hstack(
    (train_x, np.vstack(encode_cate.FractionNumber.tolist()))
)

encode_cate['UserName'] = train_sample['UserName'].apply(lambda x: sum(label_encoder_UserName.transform(np.array(x).reshape(-1,1))))
train_x = np.hstack(
    (train_x, np.vstack(encode_cate.UserName.tolist()))
)

encode_cate['CourseId'] = train_sample['CourseId'].apply(lambda x: sum(label_encoder_CourseId.transform(np.array(x).reshape(-1,1))))
train_x = np.hstack(
    (train_x, np.vstack(encode_cate.CourseId.tolist()))
)

encode_cate['ResourceSerNum'] = train_sample['ResourceSerNum'].apply(lambda x: sum(label_encoder_ResourceSerNum.transform(np.array(x).reshape(-1,1))))
train_x = np.hstack(
    (train_x, np.vstack(encode_cate.ResourceSerNum.tolist()))
)



In [18]:
train_x = np.hstack((train_x, train_num))

In [20]:
train_x.shape

(4, 438)

In [21]:
zeros = np.zeros((maxlen-encode_cate.shape[0], feature_len))
train_x = np.vstack((zeros, train_x))

In [22]:
train_x.shape

(35, 438)

In [23]:
model_sequence = Sequential()
model_sequence.add(layers.LSTM(
        batch_input_shape = (None , appt_maxlen, feature_len),
        output_dim = 128,
        dropout=0.1,
        recurrent_dropout=0.5,
        return_sequences=True,
        ))

model_sequence.add(layers.LSTM(
        output_dim = 32,
        dropout=0.1,
        recurrent_dropout=0.5,
        ))
# stateful = True 本次batch的参数返回到下一次的训练中

model_sequence.add(layers.Dense(1))

model_sequence.compile(
        optimizer = 'rmsprop',
        loss = 'mae'
        )

model_sequence.summary()

W0514 21:04:36.289557 17012 deprecation_wrapper.py:119] From C:\Users\WENDY\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

C:\Users\WENDY\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(batch_input_shape=(None, 35,..., dropout=0.1, recurrent_dropout=0.5, return_sequences=True, units=128)`
  import sys
W0514 21:04:36.359532 17012 deprecation_wrapper.py:119] From C:\Users\WENDY\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0514 21:04:36.373686 17012 deprecation_wrapper.py:119] From C:\Users\WENDY\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0514 21:04:36.523932 17012 deprecat

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 35, 128)           290304    
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 310,945
Trainable params: 310,945
Non-trainable params: 0
_________________________________________________________________


In [28]:
train_x_ = train_x.copy()
train_x_ = np.array([train_x_, train_x_])

In [29]:
train_x_.shape

(2, 35, 438)

In [ ]:
batch = 

In [30]:
model_sequence.fit(train_x_, [label_sample, label_sample])

W0514 21:08:44.609842 17012 deprecation.py:323] From C:\Users\WENDY\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0514 21:08:46.732163 17012 deprecation_wrapper.py:119] From C:\Users\WENDY\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1
2/2 [==============================] - 2s 1s/step - loss: 19.0296


In [ ]:
encode_cate = hstack((a,b, c))

In [72]:
a = np.array([[0, 1, 2],
             [3, 4, 5],
             [6, 7, 8]])

b = np.array([[0, 1, 2],
             [3, 4, 5],
             [6, 7, 8]])

c = np.array([[0, 1, 2],
             [3, 4, 5],
             [6, 7, 8]])

np.hstack((a,b, c))

array([[0, 1, 2, 0, 1, 2, 0, 1, 2],
       [3, 4, 5, 3, 4, 5, 3, 4, 5],
       [6, 7, 8, 6, 7, 8, 6, 7, 8]])

In [47]:
# Index(['PatientSerNum', 'AppointmentSerNum',  
#        'ScheduledStartTime', 'ScheduledEndTime', 
#        'ActualStartDate', 'ActualEndDate',
#        'CourseSerNum', 'PlanSerNum', 'RadiationSerNum'

feature_num = ['Scheduled_duration', 'Actual_duration', 'Actual_duration',
               'age', 'TreatmentTime_total', 'ImagesTaken_total',
               'MU_total', 'MUCoeff_total']

# RadiationId
feature_cate = ['dxt_AliasName', 'Sex', 'AliasSerNum', 
                'month', 'week', 'hour', 'DoctorSerNum', 
                'TreatmentOrientation', 'FractionNumber',
                'UserName', 'CourseId', 'ResourceSerNum']

# 因为数据缺失，所以添加新特征，表示是否是连续的序列

In [70]:
data_cate_ = series_data[feature_cate]
data_num_ = series_data[feature_num]

In [71]:
data_cate = data_cate_.copy()

In [49]:
data_cate['dxt_AliasName'] = data_cate['dxt_AliasName'].apply(lambda x: x)

,dxt_AliasName,Sex,AliasSerNum,month,week,hour,DoctorSerNum,TreatmentOrientation,FractionNumber,UserName,CourseId,ResourceSerNum
470,[GU except prostate],Male,[23],1,1,9,5246.0,[HFS],24,[poon],[C1],[95]
471,[GU except prostate],Male,[23],1,2,9,5246.0,[HFS],25,[poon],[C1],[95]
169,[METS],Male,[31],1,2,14,272.0,[HFS],1,[Chen Shi Lei],[C1],[107]
473,[GU except prostate],Male,[31],1,3,9,5246.0,[HFS],1,[Basdeo Reeta],[C1],[95]
170,[METS],Male,[23],1,3,14,272.0,[HFS],2,[Stachura Jacob],[C1],[107]
472,[GU except prostate],Male,[23],1,4,9,5246.0,[HFS],2,[Rabanal Javier],[C1],[95]
167,[METS],Male,[23],1,4,13,272.0,[HFS],3,[Lin Michael],[C1],[107]
474,[GU except prostate],Male,[23],1,5,9,5246.0,[HFS],3,[Langlais Marianne],[C1],[95]
171,[METS],Male,[23],1,5,13,272.0,[HFS],4,[nowac sydnee],[C1],[107]
173,[METS],Male,[23],1,1,13,272.0,[HFS],5,[Stachura Jacob],[C1],[107]


In [72]:
data_cate.loc[470]

dxt_AliasName           [GU except prostate]
Sex                                     Male
AliasSerNum                             [23]
month                                      1
week                                       1
hour                                       9
DoctorSerNum                            5246
TreatmentOrientation                   [HFS]
FractionNumber                            24
UserName                              [poon]
CourseId                                [C1]
ResourceSerNum                          [95]
Name: 470, dtype: object

In [79]:
label_encoder_dxt_AliasName.transform(np.array(['GU except prostate', 'METS']).reshape(-1,1))

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0.]])

In [80]:
sum(label_encoder_dxt_AliasName.transform(np.array(['GU except prostate']).reshape(-1,1)))

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [47]:
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()# sparse = False

In [53]:
data1 = data_part1.iloc[: 10000]
data2 = data_part2.iloc[: 10000]

encode_data = pd.merge(data1, data2, on = ['PatientSerNum', 'date'], how = 'inner')
encode_data.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
encode_data = encode_data[feature_cate]
print(f'The shape of encode_data is {encode_data.shape}')

The shape of encode_data is (13254, 13)


In [54]:
integer_encoded = label_encoder.fit_transform(encode_data.Sex)
integer_encoded

array([1, 1, 1, ..., 0, 0, 0])

In [58]:
inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[0, :])])
print(inverted)

['Male']


In [56]:
# one-hot encode
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
onehot_encoded

C:\Users\WENDY\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


<13254x2 sparse matrix of type '<class 'numpy.float64'>'
	with 13254 stored elements in Compressed Sparse Row format>